In [3]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
pre_dfRegNet = pd.read_csv('tableData.csv')  
# df = pre_dfRegNet[pd.notna(pre_dfRegNet['3)RegulatorGeneName'])] #dropping NaN 


- 0 `regulagtorID` = TF, node-A 
- 1 `regulaotorName` = TF's name. A-label
- 3 `regulatedId` = gene regulated. node-B 
- 4 `regulatedName` = gene's name, B-label
- 5 `funciton` = type of regylation
- 6 `confidenceLevel` = confirmed/strong/weak/unknown

Regulators and regulateds have unique IDs and there are no `Nan`s in that. But they don't have any mutual elements. But the cell below checks for mutual elements in the genes responsible for production of TFs and the ones they regulate, and there are 239 such genes. 

In [ ]:
mutualsNames = list(set(pre_dfRegNet["3)RegulatorGeneName"]).intersection(set(pre_dfRegNet["5)regulatedName"])))
print(len(mutualsNames))
print(mutualsNames)

239
['mprA', 'sgrR', 'phoP', 'mcbR', 'evgA', 'dicA', 'alaS', 'aaeR', 'gadE', 'mngR', 'cynR', 'nfeR', 'thrS', 'dpiA', 'decR', 'pnp', 'oxyR', 'rpoN', 'rplD', 'gadX', 'puuR', 'galS', 'melR', 'yeiL', 'marA', 'rstA', 'lexA', 'rprA', 'xylR', 'hfq', 'uvrY', 'hcaR', 'stpA', 'cspA', 'putA', 'uidR', 'mcaS', 'hipB', 'mlc', 'narL', 'murR', 'zntR', 'creB', 'cusR', 'ygiV', 'ibpA', 'cdaR', 'hdfR', 'allS', 'ttdR', 'rcsB', 'fliZ', 'lrp', 'plaR', 'nac', 'tdcR', 'lsrR', 'gutM', 'fur', 'rspR', 'iclR', 'exuR', 'lldR', 'fhlA', 'gadY', 'uhpU', 'uxuR', 'ryhB', 'mhpR', 'hicB', 'cytR', 'asnC', 'metR', 'zraR', 'paaX', 'sdiA', 'csqR', 'lacI', 'cbl', 'purR', 'citR', 'mqsA', 'yciT', 'soxS', 'rplA', 'ydaT', 'gcvA', 'cpxR', 'rob', 'hyfR', 'csrA', 'fimR2', 'higA', 'selB', 'metJ', 'rpsH', 'uhpA', 'rpsD', 'bolA', 'rplM', 'dgoR', 'leuO', 'pdhR', 'trpR', 'tyrR', 'soxR', 'arcZ', 'marR', 'malI', 'cyaR', 'srsR', 'cra', 'sgrS', 'cecR', 'narP', 'torR', 'hns', 'yafC', 'mazE', 'ydeO', 'srlR', 'agaR', 'treR', 'matA', 'nhaR', 'frm

In [11]:
regnet = pre_dfRegNet.loc[:, ['3)RegulatorGeneName', '5)regulatedName']]
g  = nx.digraph()
g.add_edges_from(regnet.values)

TypeError: 'module' object is not callable

# 1. Basic Statistics

In [ ]:
pre_dfRegNet

,1)regulatorId,2)regulatorName,3)RegulatorGeneName,4)regulatedId,5)regulatedName,6)function,7)confidenceLevel
0,RDBECOLICNC00063,ppGpp,NaN,RDBECOLIGNC00006,hisM,-,W
1,RDBECOLICNC00063,ppGpp,NaN,RDBECOLIGNC00030,agp,+,W
2,RDBECOLICNC00063,ppGpp,NaN,RDBECOLIGNC00039,cysQ,+,W
3,RDBECOLICNC00063,ppGpp,NaN,RDBECOLIGNC00065,argI,+,W
4,RDBECOLICNC00063,ppGpp,NaN,RDBECOLIGNC00067,argS,-,W
...,...,...,...,...,...,...,...
7278,RDBECOLITFC00239,DicD,dicD,RDBECOLIGNC00222,dicC,-,W
7279,RDBECOLITFC00240,YdiP,ydiP,RDBECOLIGNC00162,cspA,+,W
7280,RDBECOLITFC00240,YdiP,ydiP,RDBECOLIGNC02067,cspE,+,W
7281,RDBECOLITFC00241,YdaT,ydaT,RDBECOLIGNC00810,rcsA,+,W


In [53]:
TFs = np.unique(rgnetwork[:, 2]) #number of type-A nodes(TFs)
GNs = np.unique(rgnetwork[:, 3])

rednw = rgnetwork[:, 0]

In [54]:
TFs.shape

(316,)

In [52]:
TFs.shape

(319,)

In [ ]:
def Est_inDegree(rgnetwork)

set()
